# Sentiment Analysis through LSTMs

### Introduction to NLP

Language bonds humans together. Language acts as a tool for us to convey our thoughts and feelings to another person and with the help of the same tool we are able to understand their thoughts and feelings as well. Most of us start talking from around 18 months old to 2 years. It is not yet completely understood how the human brain is able to grasp such vast amounts of knowledge at such a tender age. But, it has been found that most of the language processing functions happen within the cerebral cortex of the brain.

## Motivation Behind Sentiment Analysis

Humans ourselves are not able to understand how exactly language is processed by our brains. So, is it possible for us to teach a machine to learn our language?? Yes, through extensive research, a lot of methods have been developed that could help machines understand our languages. NLP or Natural Language Processing is the field of study that focuses on the interactions between human language and computers. One subproblem of NLP is sentiment analysis, i.e classifying a statement as positive or negative. What is the use of classifying a statement as positive or negative?? Let’s take an example of Amazon website. On Amazon, it’s users can leave a comment about a product stating whether it was good, bad or it could even be neutral. Now, using a human to read all the comments and obtaining the overall customer feedback on the product would be expensive and time-consuming. Enter our machine learning model. The machine learning model can churn through a vast amount of data, making inferences and classifying the comment. Using this ML model, Amazon can better its products through the customer reviews which would bring in more revenue for the company.

Sentiment analysis isn’t as straightforward as it may seem. If you think that the comments which contain the words “good”, “awesome”, etc can be classified as a positive comment and the comments which the words “bad”, “miserable” etc can be classified as a negative comment, think again. E.x: “Completely lacking in good taste” and “Good for a quick meal but nothing special” represent a negative and neutral feedback respectively even though they have the word “good” in them. Therefore, as I mentioned the task may not be as easy as it may seem. Let’s move on to the data we will be working with.

# Dataset

We are going to look at comments from Amazon products reviews, IMDB movie criticisms and Yelp reviews to build our sentiment analysis model. The data can be downloaded from this link https://www.kaggle.com/marklvl/sentiment-labelled-sentences-data-set/data. All the data has been annotated, 0 represents a negative feedback and 1 represents a positive feedback. 

# Code

In [6]:
#dataset reading

with open("data/amazon_cells_labelled.txt") as f1:
    lines = f1.readlines()

with open("data/imdb_labelled.txt") as f1:
    temp = f1.readlines()
    lines=lines+temp

with open("data/yelp_labelled.txt") as f1:
    temp = f1.readlines()
    lines=lines+temp

The data is present in different text files. We open each of this file and read all the lines of text which also includes the labels for each text. We then store them in a list called “lines”.

In [7]:
x = []
y = []
for value in lines:
    temp = value.split('\t')
    x.append(temp[0])
    temp[1].replace('\n','')
    y.append(int(temp[1]))

At each line in the dataset, we have the text and four character spaces after, we have the label for that text(0 or 1). Therefore, we take the first part which has the text and add it to our features(x) and then we take the label, which has ‘\n’ at the end. So, its removed and then added to our list of labels(y).

In [8]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=2500,split=' ')
tokenizer.fit_on_texts(x)

Keras has a built-in API that makes preparing text for computation easier. The Tokenizer class has 4 attributes which we can use to prepare our features.

In [9]:
tokenizer = Tokenizer()
texts = ["The sun is shining in June!","September is grey.","Life is beautiful in August.","I like it","This and other things?"]
tokenizer.fit_on_texts(texts)
print(tokenizer.word_index)
tokenizer.texts_to_sequences(["June is beautiful and I like it!"])

{'is': 1, 'in': 2, 'the': 3, 'sun': 4, 'shining': 5, 'june': 6, 'september': 7, 'grey': 8, 'life': 9, 'beautiful': 10, 'august': 11, 'i': 12, 'like': 13, 'it': 14, 'this': 15, 'and': 16, 'other': 17, 'things': 18}


[[6, 1, 10, 16, 12, 13, 14]]

Tokenizer assigns index values to each word in the sentence and a new sentence can represented using this index values. Since the text corpus we are using contains a huge number of different words, we set a upper bound and use only the most recurring 2500 words.

In [10]:
from keras.preprocessing.sequence import pad_sequences

X = tokenizer.texts_to_sequences(x)
X = pad_sequences(X)

We now convert the text into these number sequences as shown above and pad the sequence of numbers. Since sentences can be of different length, their sequence length will also be different. Therefore, pad_sequences finds the longest sentence and pads all other statements with 0 to match that length.

In [12]:
## Pad Sequences Example
pad_sequences([[1, 2, 3], [3, 4, 5, 6], [7, 8]])


array([[0, 1, 2, 3],
       [3, 4, 5, 6],
       [0, 0, 7, 8]])

In [13]:
import numpy as np 
from sklearn.model_selection import train_test_split

Y = []
for val in y:
    if(val == 0):
        Y.append([1,0])
    else:
        Y.append([0,1])
Y = np.array(Y)

x_train, x_test, y_train, y_test = train_test_split(X,Y,train_size=0.8)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


We convert the labels into one hot encoding. This helps the LSTM network in predicting the label for the text. Now, we have prepared our text data, we can split it into training and test samples. 80% of the data is used for training and 20% is kept aside for testing the model.

In [14]:
import keras 
from keras.layers import Embedding, LSTM, Dense
from keras.models import Sequential

model = Sequential()
model.add(Embedding(2500,128,input_length=X.shape[1],dropout=0.2))
model.add(LSTM(300, dropout_U=0.2,dropout_W=0.2))
model.add(Dense(2,activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,optimizer='adam',metrics=['accuracy'])

model.fit(x_train,y_train,epochs=10,verbose=2,batch_size=32)

print(model.evaluate(x_test,y_test)[1])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(300, dropout=0.2, recurrent_dropout=0.2)`
  import sys


Epoch 1/10
 - 14s - loss: 0.6910 - acc: 0.5304
Epoch 2/10
 - 10s - loss: 0.6856 - acc: 0.5542
Epoch 3/10
 - 10s - loss: 0.6845 - acc: 0.5483
Epoch 4/10
 - 10s - loss: 0.6832 - acc: 0.5613
Epoch 5/10
 - 9s - loss: 0.6820 - acc: 0.5592
Epoch 6/10
 - 8s - loss: 0.6819 - acc: 0.5454
Epoch 7/10
 - 9s - loss: 0.6799 - acc: 0.5554
Epoch 8/10
 - 9s - loss: 0.6790 - acc: 0.5633
Epoch 9/10
 - 10s - loss: 0.6765 - acc: 0.5517
Epoch 10/10
 - 10s - loss: 0.6771 - acc: 0.5546
600/600 [==============================] - 1s 2ms/step
0.551666665871938


We now build the model, compile it, train it and test it. The model has an embedding layer. The input sequences are sparse representations of the text because the vocabulary would be huge and a given word would be represented by a large vector. It would be easier for the network to predict if we are able to build some dense representation of the sequence. The word embeddings/dense representation for the 2500 words are derived from training the model by the embedding layer. We then add LSTM and Dense layers to the model. The LSTM cell is responsible for making contextual inferences and aids in predicting whether a sentence is positive or not. The Dense layer outputs the probabilities for each class.